# Import libraries

In [1]:
import numpy as np
import os
import torch
import torchvision
from torchvision import datasets, models, transforms
from pathlib import Path

%matplotlib inline

In [2]:
# Check presence of GPU
is_gpu_avalaible = torch.cuda.is_available()
print(is_gpu_avalaible)

False


# Load the dataset

In [3]:
data = "dataset/eczema/"

train_dir = os.path.join(data, "train/")
test_dir = os.path.join(data, "test/")
val_dir = os.path.join(data, "val/")

In [4]:
classes = ["Atopic dermatitis", "Contact dermatitis", "Dyshidrotic eczema","Hand eczema","Neurodermatitis","Nummular eczema","Stasis dermatitis"]

# Transform the dataset

In [5]:
data_transformer = transforms.Compose([transforms.RandomResizedCrop(224), transforms.ToTensor()])

In [11]:
train_data = datasets.ImageFolder(train_dir, transform=data_transformer)
val_data = datasets.ImageFolder(val_dir, transform=data_transformer)
test_data = datasets.ImageFolder(test_dir, transform=data_transformer)